In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Get an active session from snowflake.snowpark
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from snowflake.snowpark.functions import col, hour
import warnings
warnings.filterwarnings('ignore')


TIME_COL = "TIME_COL"   
PARENT_COL = "PARENT_NAME"  
TABLE_NAME = "vitalise_data_light_01"
MIN_VARIANCE_EXPLAINED = 0.90  
MIN_OCCURRENCE_THRESHOLD = 0.001

class PCAAnalyzer:
    """PCA analysis for sparse recording hours"""
    
    def __init__(self, min_variance=0.90, min_occurrence=0.001):
        self.min_variance = min_variance
        self.min_occurrence = min_occurrence
        self.scaler = StandardScaler()
        self.pca = None
        self.variance_selector = None
        self.actual_hours = None  # Will store actual available hours
        
    def discover_actual_recording_hours(self, session):
        """Discover which hours actually have data"""
        print("DISCOVERING ACTUAL RECORDING HOURS")
        print("*" * 50)
        
        # Check which hours have data
        hour_check = session.sql(f"""
            SELECT HOUR({TIME_COL}) as hour, COUNT(*) as records 
            FROM {TABLE_NAME} 
            WHERE {PARENT_COL} IS NOT NULL
            GROUP BY HOUR({TIME_COL}) 
            ORDER BY hour
        """).to_pandas()
        
        self.actual_hours = sorted(hour_check['HOUR'].tolist())
        
        print(f"Actual Recording Hours Found:")
        for _, row in hour_check.iterrows():
            print(f"   • Hour {row['HOUR']:2d}: {row['RECORDS']:,} records")
            
        print(f"Analysis Summary:")
        print(f"Total hours with data: {len(self.actual_hours)}")
        print(f"Hour range: {min(self.actual_hours)}-{max(self.actual_hours)}")
        print(f"Missing hours: {set(range(24)) - set(self.actual_hours)}")
        
        # Identify gaps
        missing_hours = sorted(set(range(24)) - set(self.actual_hours))
        if missing_hours:
            gaps = []
            current_gap = [missing_hours[0]]
            for i in range(1, len(missing_hours)):
                if missing_hours[i] == missing_hours[i-1] + 1:
                    current_gap.append(missing_hours[i])
                else:
                    gaps.append(current_gap)
                    current_gap = [missing_hours[i]]
            gaps.append(current_gap)
            
            print(f"Recording gaps:")
            for gap in gaps:
                if len(gap) == 1:
                    print(f"- Hour {gap[0]}")
                else:
                    print(f"- Hours {gap[0]}-{gap[-1]} ({len(gap)} hours)")
        
        return self.actual_hours, hour_check
    
    def load_and_prepare_sparse_data(self, session):
        """Load data only for actual recording hours"""
        
        # Discover actual hours first
        self.actual_hours, hour_stats = self.discover_actual_recording_hours(session)
        
        # Load data with hour filter
        vitalise_sp = session.table(TABLE_NAME)
        vitalise_sp = vitalise_sp.filter(col(PARENT_COL).is_not_null())
        vitalise_sp = vitalise_sp.with_column("HOUR", hour(col(TIME_COL)))
        
        # KEY FIX: Only include actual recording hours
        vitalise_sp = vitalise_sp.filter(col("HOUR").isin(self.actual_hours))
        
        agg_sp = vitalise_sp.group_by("HOUR", PARENT_COL).count()
        agg_df = agg_sp.to_pandas()
        
        print(f"Data Overview:")
        print(f"Records loaded: {len(agg_df):,}")
        print(f"Hours included: {len(self.actual_hours)} ({self.actual_hours})")
        print(f"Sound classes: {agg_df[PARENT_COL].nunique()}")
        
        return agg_df
    
    def comprehensive_class_selection(self, agg_df, parent_col):
        """Smart class selection adapted for sparse hours"""
        print(f"SOUND CLASS SELECTION (Sparse Hours)")
        
        print(f"Analysing {agg_df[parent_col].nunique()} sound classes across {len(self.actual_hours)} hours...")
        
        class_stats = agg_df.groupby(parent_col)['COUNT'].agg([
            'sum', 'mean', 'std', 'count', 'min', 'max'
        ]).round(2)
        
        total_activity = agg_df['COUNT'].sum()
        class_stats['percentage'] = (class_stats['sum'] / total_activity * 100).round(3)
        class_stats['cv'] = (class_stats['std'] / class_stats['mean']).round(3)
        
        print(f"Applying Selection Criteria:")
        
        # Criterion 1: Minimum occurrence (same)
        occurrence_filter = class_stats['percentage'] >= (self.min_occurrence * 100)
        print(f"Occurrence ≥{self.min_occurrence*100}%: {occurrence_filter.sum()} classes")
        
        # Criterion 2: Present in multiple hours (adapted for fewer hours)
        temporal_spread = agg_df.groupby(parent_col)['HOUR'].nunique()
        min_hours = max(2, len(self.actual_hours) // 3)  # At least 1/3 of available hours
        spread_filter = temporal_spread >= min_hours
        print(f"Temporal spread ≥{min_hours} hours: {spread_filter.sum()} classes")
        
        # Criterion 3: Some temporal variation (same)
        variation_filter = class_stats['cv'] > 0.1
        print(f"Temporal variation (CV>0.1): {variation_filter.sum()} classes")
        
        # Combine criteria
        selected_classes = class_stats[
            occurrence_filter & 
            spread_filter.reindex(class_stats.index, fill_value=False) &
            variation_filter
        ].sort_values('sum', ascending=False)
        
        print(f"FINAL SELECTION: {len(selected_classes)} classes")
        print(f"Coverage: {selected_classes['percentage'].sum():.1f}% of total activity")
        
        return selected_classes, class_stats
    
    def create_feature_matrix(self, agg_df, selected_class_names):
        print(f"CREATING FEATURE MATRIX")
        
        # Filter to selected classes
        filtered_df = agg_df[agg_df[PARENT_COL].isin(selected_class_names)]
        
        # Create pivot - only actual hours, no padding!
        df = filtered_df.pivot(index="HOUR", columns=PARENT_COL, values="COUNT").fillna(0)
        
        
        print(f"Feature Matrix:")
        print(f"Shape: {df.shape} (Hours × Sound Classes)")
        print(f"Hours included: {sorted(df.index.tolist())}")
        print(f"Sparsity: {(df == 0).mean().mean():.1%} (real sparsity)")
        
        return df
    
    def apply_pca(self, df):
        """Apply PCA to real data matrix"""
        
        print(f"Input matrix: {df.shape[0]} hours × {df.shape[1]} sound classes")
        
        # Remove zero-variance features
        self.variance_selector = VarianceThreshold(threshold=0)
        df_cleaned = pd.DataFrame(
            self.variance_selector.fit_transform(df),
            columns=df.columns[self.variance_selector.get_support()],
            index=df.index
        )
        
        print(f"Features after variance filtering: {df_cleaned.shape[1]}")
        
        # Standardize
        df_scaled = self.scaler.fit_transform(df_cleaned)
        
        # Apply PCA
        self.pca = PCA()
        pca_components = self.pca.fit_transform(df_scaled)
        
        # Determine optimal components
        self.optimal_components = self._determine_optimal_components()
        
        # Create results
        pca_columns = [f'PC{i+1}' for i in range(self.optimal_components)]
        pca_results = pd.DataFrame(
            pca_components[:, :self.optimal_components],
            columns=pca_columns,
            index=df.index
        )
        
        loadings = pd.DataFrame(
            self.pca.components_[:self.optimal_components].T,
            columns=pca_columns,
            index=df_cleaned.columns
        )
        
        print(f"PCA Results:")
        print(f"Optimal components: {self.optimal_components}")
        print(f"Variance explained: {self.pca.explained_variance_ratio_[:self.optimal_components].sum():.1%}")
        
        return pca_results, loadings, df_cleaned
    
    def _determine_optimal_components(self):
        """Determine optimal components for sparse data"""
        cumsum_var = np.cumsum(self.pca.explained_variance_ratio_)
        var_threshold_comps = np.argmax(cumsum_var >= self.min_variance) + 1
        kaiser_comps = np.sum(self.pca.explained_variance_ > 1)
        
        # For sparse data, be more conservative
        optimal = min(max(var_threshold_comps, kaiser_comps), len(self.actual_hours) - 1)
        
        print(f"Component Selection:")
        print(f"Variance threshold ({self.min_variance:.0%}): {var_threshold_comps}")
        print(f"Kaiser criterion (λ>1): {kaiser_comps}")
        print(f"Selected (conservative): {optimal}")
        
        return optimal
    
    def cluster_data(self, pca_results):
        """Cluster the PCA results"""
        
        # Find optimal clusters for sparse data
        max_k = min(6, len(self.actual_hours) // 2)  # Conservative for sparse data
        
        silhouette_scores = []
        k_range = range(2, max_k + 1)
        
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(pca_results)
            score = silhouette_score(pca_results, labels)
            silhouette_scores.append(score)
        
        optimal_k = k_range[np.argmax(silhouette_scores)]
        best_score = max(silhouette_scores)
        
        # Final clustering
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(pca_results)
        
        print(f"Clustering Results:")
        print(f"Optimal clusters: {optimal_k}")
        print(f"Silhouette score: {best_score:.3f}")
        print(f"Hours per cluster: {pd.Series(cluster_labels).value_counts().sort_index().tolist()}")
        
        # Interpret clusters
        self._interpret_sparse_clusters(cluster_labels, pca_results)
        
        return cluster_labels, optimal_k, best_score
    
    def _interpret_sparse_clusters(self, cluster_labels, pca_results):
        """Interpret clusters for sparse temporal data"""
        
        for cluster_id in sorted(set(cluster_labels)):
            mask = cluster_labels == cluster_id
            cluster_hours = np.array(self.actual_hours)[mask]
            
            print(f"Cluster {cluster_id} ({len(cluster_hours)} hours):")
            print(f"Hours: {sorted(cluster_hours.tolist())}")
            
            # Characterize time periods
            morning_hours = [h for h in cluster_hours if 6 <= h <= 10]
            afternoon_hours = [h for h in cluster_hours if 15 <= h <= 17]
            evening_hours = [h for h in cluster_hours if 19 <= h <= 20]
            
            characteristics = []
            if morning_hours:
                characteristics.append(f"Morning ({len(morning_hours)} hours)")
            if afternoon_hours:
                characteristics.append(f"Afternoon ({len(afternoon_hours)} hours)")
            if evening_hours:
                characteristics.append(f"Evening ({len(evening_hours)} hours)")
            
            print(f"Time periods: {', '.join(characteristics) if characteristics else 'Mixed'}")
    
    def visualize_results(self, pca_results, cluster_labels, loadings):
        """Visualize analysis results"""

        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. PCA scatter plot
        scatter = axes[0,0].scatter(pca_results.iloc[:,0], pca_results.iloc[:,1], 
                                   c=cluster_labels, cmap='viridis', s=100, alpha=0.8)
        axes[0,0].set_xlabel(f'PC1 ({self.pca.explained_variance_ratio_[0]:.1%})')
        axes[0,0].set_ylabel(f'PC2 ({self.pca.explained_variance_ratio_[1]:.1%})')
        axes[0,0].set_title('PCA Scatterplot of Daily Hours')
        
        # Add hour labels
        for i, hour in enumerate(pca_results.index):
            axes[0,0].annotate(f'{hour:02d}', (pca_results.iloc[i,0], pca_results.iloc[i,1]),
                              xytext=(3, 3), textcoords='offset points', fontweight='bold')
        
        # 2. Explained variance
        n_components = min(8, len(self.pca.explained_variance_ratio_))
        axes[0,1].bar(range(1, n_components + 1), 
                     self.pca.explained_variance_ratio_[:n_components])
        axes[0,1].set_xlabel('Component')
        axes[0,1].set_ylabel('Explained Variance')
        axes[0,1].set_title('PCA Explained Variance')
        
        # 3. Temporal pattern
        colors = ['red', 'blue', 'green', 'purple', 'orange']
        for cluster_id in sorted(set(cluster_labels)):
            mask = cluster_labels == cluster_id
            cluster_hours = np.array(self.actual_hours)[mask]
            cluster_activities = [1] * len(cluster_hours)  # Placeholder
            
            axes[1,0].scatter(cluster_hours, cluster_activities, 
                             color=colors[cluster_id % len(colors)], 
                             label=f'Cluster {cluster_id}', s=100, alpha=0.8)
        
        axes[1,0].set_xlabel('Hour of Day')
        axes[1,0].set_ylabel('Cluster Assignment')
        axes[1,0].set_title('Sparse Temporal Pattern')
        axes[1,0].set_xticks(range(0, 24, 2))
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
        
        # 4. Top loadings
        top_loadings = loadings['PC1'].abs().nlargest(10)
        axes[1,1].barh(range(len(top_loadings)), top_loadings.values)
        axes[1,1].set_yticks(range(len(top_loadings)))
        axes[1,1].set_yticklabels(top_loadings.index, fontsize=8)
        axes[1,1].set_title('Top PC1 Loadings')
        
        plt.tight_layout()
        plt.show()

def run_analysis(session):
    """Run the analysis without padding"""
    
    # Initialise analyzer
    analyzer = PCAAnalyzer(min_variance=0.90, min_occurrence=0.001)
    
    # Step 1: Load data
    agg_df = analyzer.load_and_prepare_sparse_data(session)
    
    # Step 2: Smart class selection
    selected_classes, all_class_stats = analyzer.comprehensive_class_selection(agg_df, PARENT_COL)
    
    # Step 3: Create feature matrix
    selected_class_names = selected_classes.index.tolist()
    df = analyzer.create_feature_matrix(agg_df, selected_class_names)
    
    # Step 4: Apply PCA
    pca_results, loadings, df_cleaned = analyzer.apply_pca(df)
    
    # Step 5: Cluster data
    cluster_labels, optimal_k, silhouette_score = analyzer.cluster_data(pca_results)
    
    # Step 6: Visualize
    analyzer.visualize_results(pca_results, cluster_labels, loadings)

    
    print(f"ANALYSIS COMPLETE!")
    print(f"Real hours analyzed: {len(analyzer.actual_hours)}")
    print(f"Clusters found: {optimal_k}")
    print(f"Clustering quality: {silhouette_score:.3f}")
    print(f"Based on REAL patterns, not artificial padding!")
    
    return analyzer, pca_results, cluster_labels, loadings

# Run the analysis:
analyzer, pca_results, clusters, loadings = run_analysis(session)

In [ ]:
# Inspect PCA Components


# 1. Variance explained by each component
explained_variance = analyzer.pca.explained_variance_ratio_
print("Explained variance by component:")
for i, var in enumerate(explained_variance[:10], 1):  # first 10 PCs
    print(f"PC{i}: {var:.2%}")

# 2. Top loadings for PC1 and PC2
print("Top contributing sound classes for PC1:")
print(loadings['PC1'].sort_values(ascending=False).head(10))

print("Top contributing sound classes for PC2:")
print(loadings['PC2'].sort_values(ascending=False).head(10))

# 3. Bottom loadings (negative contributions)
print("Negative contributors to PC1:")
print(loadings['PC1'].sort_values().head(10))

print("Negative contributors to PC2:")
print(loadings['PC2'].sort_values().head(10))


In [ ]:
"""
POST-PCA ANALYSIS SUITE
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from snowflake.snowpark.functions import col, hour
import warnings
warnings.filterwarnings('ignore')


# 1. DEEP CLUSTER ANALYSIS


class PostPCAAnalyzer:
    """Advanced analysis after PCA clustering"""
    
    def __init__(self, analyzer, pca_results, clusters, loadings, session):
        self.analyzer = analyzer
        self.pca_results = pca_results
        self.clusters = clusters
        self.loadings = loadings
        self.session = session
        self.cluster_profiles = {}
        
    def deep_cluster_analysis(self):
        """Comprehensive cluster characterization"""

        # Create cluster profiles
        for cluster_id in sorted(set(self.clusters)):
            mask = self.clusters == cluster_id
            cluster_hours = np.array(self.analyzer.actual_hours)[mask]
            cluster_pca = self.pca_results[mask]
            
            profile = {
                'hours': cluster_hours,
                'size': len(cluster_hours),
                'pca_mean': cluster_pca.mean(axis=0),
                'pca_std': cluster_pca.std(axis=0),
                'time_periods': self._categorize_time_periods(cluster_hours)
            }
            
            self.cluster_profiles[cluster_id] = profile
            
            print(f"CLUSTER {cluster_id} DETAILED PROFILE:")
            print(f"Hours: {sorted(cluster_hours.tolist())}")
            print(f"Size: {profile['size']} hours ({profile['size']/len(self.analyzer.actual_hours)*100:.1f}%)")
            print(f"Time periods: {profile['time_periods']}")
            print(f"PC1 mean: {profile['pca_mean'][0]:.3f} ± {profile['pca_std'][0]:.3f}")
            print(f"PC2 mean: {profile['pca_mean'][1]:.3f} ± {profile['pca_std'][1]:.3f}")
        
        return self.cluster_profiles
    
    def _categorize_time_periods(self, hours):
        """Categorize hours into time periods"""
        periods = {
            'Dawn': [h for h in hours if 6 <= h <= 7],
            'Morning': [h for h in hours if 8 <= h <= 11],
            'Midday': [h for h in hours if 12 <= h <= 14],
            'Afternoon': [h for h in hours if 15 <= h <= 17],
            'Evening': [h for h in hours if 18 <= h <= 20],
            'Night': [h for h in hours if 21 <= h <= 23]
        }
        
        return {period: len(period_hours) for period, period_hours in periods.items() if period_hours}
    
    def analyze_driving_sound_classes(self):
        """Identify which sound classes distinguish clusters"""
        
        # Get original data matrix
        original_data = self._reconstruct_original_data()
        
        # Statistical tests between clusters
        cluster_differences = {}
        
        for sound_class in original_data.columns:
            cluster_0_data = original_data.loc[self.clusters == 0, sound_class]
            cluster_1_data = original_data.loc[self.clusters == 1, sound_class]
            
            # Mann-Whitney U test (non-parametric)
            statistic, p_value = stats.mannwhitneyu(cluster_0_data, cluster_1_data, 
                                                   alternative='two-sided')
            
            # Effect size (Cohen's d)
            pooled_std = np.sqrt(((len(cluster_0_data)-1)*cluster_0_data.var() + 
                                (len(cluster_1_data)-1)*cluster_1_data.var()) / 
                               (len(cluster_0_data) + len(cluster_1_data) - 2))
            
            cohens_d = abs(cluster_0_data.mean() - cluster_1_data.mean()) / pooled_std
            
            cluster_differences[sound_class] = {
                'p_value': p_value,
                'effect_size': cohens_d,
                'cluster_0_mean': cluster_0_data.mean(),
                'cluster_1_mean': cluster_1_data.mean(),
                'difference': cluster_1_data.mean() - cluster_0_data.mean()
            }
        
        # Sort by effect size
        sorted_diffs = sorted(cluster_differences.items(), 
                            key=lambda x: x[1]['effect_size'], reverse=True)
        
        print(f"TOP 10 DISCRIMINATING SOUND CLASSES:")
        print(f"{'Sound Class':<25} {'Effect Size':<12} {'P-value':<10} {'Cluster Preference'}")

        
        for sound_class, stats_dict in sorted_diffs[:10]:
            p_val = stats_dict['p_value']
            effect = stats_dict['effect_size']
            
            # Determine which cluster has higher activity
            if stats_dict['difference'] > 0:
                preference = f"Cluster 1 ({stats_dict['cluster_1_mean']:.1f} vs {stats_dict['cluster_0_mean']:.1f})"
            else:
                preference = f"Cluster 0 ({stats_dict['cluster_0_mean']:.1f} vs {stats_dict['cluster_1_mean']:.1f})"
            
            significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
            
            print(f"{sound_class:<25} {effect:.3f}{significance:<9} {p_val:<10.4f} {preference}")
        
        return cluster_differences
    
    def _reconstruct_original_data(self):
        """Reconstruct original data matrix from analyzer"""
        
        try:
            vitalise_sp = self.session.table("vitalise_data_light_01")
            vitalise_sp = vitalise_sp.filter(col("PARENT_NAME").is_not_null())
            vitalise_sp = vitalise_sp.with_column("HOUR", hour(col("TIME_COL")))
            vitalise_sp = vitalise_sp.filter(col("HOUR").isin(self.analyzer.actual_hours))
            
            agg_sp = vitalise_sp.group_by("HOUR", "PARENT_NAME").count()
            agg_df = agg_sp.to_pandas()
            
            # Get the same selected classes as before
            selected_classes = self.loadings.index.tolist()
            filtered_df = agg_df[agg_df["PARENT_NAME"].isin(selected_classes)]
            
            df = filtered_df.pivot(index="HOUR", columns="PARENT_NAME", values="COUNT").fillna(0)
            
            return df
        except Exception as e:
            print(f"Warning: Could not reconstruct original data. Using PCA approximation. Error: {e}")
            approx_data = np.dot(self.pca_results.values, self.loadings.T.values)
            
            return pd.DataFrame(
                approx_data, 
                index=self.pca_results.index,
                columns=self.loadings.index
            )


def analyze_temporal_transitions(analyzer, clusters):
    """Analyse transitions between acoustic states"""
    
    hours = np.array(analyzer.actual_hours)
    
    # Find transitions
    transitions = []
    for i in range(len(clusters) - 1):
        if hours[i+1] == hours[i] + 1:  # Consecutive hours
            if clusters[i] != clusters[i+1]:
                transitions.append({
                    'from_hour': hours[i],
                    'to_hour': hours[i+1],
                    'from_cluster': clusters[i],
                    'to_cluster': clusters[i+1]
                })
    
    print(f"CLUSTER TRANSITIONS FOUND: {len(transitions)}")
    for trans in transitions:
        print(f"Hour {trans['from_hour']} → {trans['to_hour']}: "
              f"Cluster {trans['from_cluster']} → {trans['to_cluster']}")
    
    # Transition probabilities
    if len(transitions) > 0:
        transition_matrix = np.zeros((2, 2))
        for trans in transitions:
            transition_matrix[trans['from_cluster'], trans['to_cluster']] += 1
        
        # Normalize
        row_sums = transition_matrix.sum(axis=1)
        transition_matrix = transition_matrix / row_sums[:, np.newaxis]
        
        print(f"TRANSITION PROBABILITY MATRIX:")
        print(f"     Cluster 0  Cluster 1")
        print(f"C0:    {transition_matrix[0,0]:.3f}     {transition_matrix[0,1]:.3f}")
        print(f"C1:    {transition_matrix[1,0]:.3f}     {transition_matrix[1,1]:.3f}")
    
    return transitions


# 3. ACOUSTIC DIVERSITY INDICES


def calculate_acoustic_diversity_indices(session, analyzer):
    """Calculate standard acoustic ecology indices"""
    
    # Load hourly data
    diversity_data = []
    
    try:
        for hour in analyzer.actual_hours:
            # Get data for this hour
            hour_query = f"""
            SELECT PARENT_NAME, COUNT(*) as activity
            FROM vitalise_data_light_01 
            WHERE PARENT_NAME IS NOT NULL 
            AND HOUR(TIME_COL) = {hour}
            GROUP BY PARENT_NAME
            """
            
            hour_df = session.sql(hour_query).to_pandas()
            
            if len(hour_df) > 0:
                # Shannon Diversity Index
                total_activity = hour_df['ACTIVITY'].sum()
                proportions = hour_df['ACTIVITY'] / total_activity
                proportions = proportions[proportions > 0]  # Remove zeros for log calculation
                shannon_div = -np.sum(proportions * np.log(proportions))
                
                # Simpson's Diversity Index
                simpson_div = 1 - np.sum(proportions ** 2)
                
                # Species richness (number of sound classes)
                richness = len(hour_df)
                
                # Evenness
                max_shannon = np.log(richness) if richness > 1 else 1
                evenness = shannon_div / max_shannon if max_shannon > 0 else 0
                
                diversity_data.append({
                    'hour': hour,
                    'shannon_diversity': shannon_div,
                    'simpson_diversity': simpson_div,
                    'richness': richness,
                    'evenness': evenness,
                    'total_activity': total_activity
                })
        
        if diversity_data:
            diversity_df = pd.DataFrame(diversity_data)
            
            print(f"ACOUSTIC DIVERSITY SUMMARY:")
            print(f"Shannon Diversity: {diversity_df['shannon_diversity'].mean():.3f} ± {diversity_df['shannon_diversity'].std():.3f}")
            print(f"Simpson Diversity: {diversity_df['simpson_diversity'].mean():.3f} ± {diversity_df['simpson_diversity'].std():.3f}")
            print(f"Average Richness: {diversity_df['richness'].mean():.1f} ± {diversity_df['richness'].std():.1f}")
            print(f"Average Evenness: {diversity_df['evenness'].mean():.3f} ± {diversity_df['evenness'].std():.3f}")
            
            return diversity_df
        else:
            print("Warning: No diversity data calculated")
            return None
            
    except Exception as e:
        print(f"Warning: Could not calculate diversity indices. Error: {e}")
        return None

# 4. PREDICTIVE MODELING

def build_cluster_prediction_model(pca_results, clusters, analyzer):
    """Build model to predict cluster membership"""
    
    # Prepare features
    X = pca_results.values
    y = clusters
    
    # Add temporal features
    hours = np.array(analyzer.actual_hours).reshape(-1, 1)
    
    # Create cyclical time features
    hour_sin = np.sin(2 * np.pi * hours / 24)
    hour_cos = np.cos(2 * np.pi * hours / 24)
    
    # Combine features
    X_enhanced = np.hstack([X, hour_sin, hour_cos])
    feature_names = (list(pca_results.columns) + ['hour_sin', 'hour_cos'])
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_enhanced, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Train Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Cross-validation
    cv_scores = cross_val_score(rf_model, X_enhanced, y, cv=5)
    
    # Predictions
    y_pred = rf_model.predict(X_test)
    
    print(f"MODEL PERFORMANCE:")
    print(f"Cross-validation accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    print(f"Test accuracy: {(y_pred == y_test).mean():.3f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"TOP FEATURE IMPORTANCES:")
    for _, row in feature_importance.head(8).iterrows():
        print(f"   • {row['feature']}: {row['importance']:.3f}")
    
    return rf_model, feature_importance

# 5. VISUALIZATION SUITE


def create_comprehensive_visualizations(analyzer, pca_results, clusters, loadings, diversity_df=None):
    """Create visualization suite"""
    
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Enhanced PCA plot with annotations
    ax1 = plt.subplot(3, 4, 1)
    colors = ['red', 'blue', 'green', 'purple']
    
    for cluster_id in sorted(set(clusters)):
        mask = clusters == cluster_id
        cluster_hours = np.array(analyzer.actual_hours)[mask]
        cluster_pca = pca_results[mask]
        
        ax1.scatter(cluster_pca.iloc[:,0], cluster_pca.iloc[:,1], 
                   c=colors[cluster_id], label=f'Cluster {cluster_id}', 
                   s=100, alpha=0.7, edgecolors='black')
        
        # Add hour labels
        for i, hour in enumerate(cluster_hours):
            ax1.annotate(f'{hour:02d}', 
                        (cluster_pca.iloc[i,0], cluster_pca.iloc[i,1]),
                        xytext=(3, 3), textcoords='offset points', 
                        fontweight='bold', fontsize=8)
    
    ax1.set_xlabel(f'PC1 ({analyzer.pca.explained_variance_ratio_[0]:.1%})')
    ax1.set_ylabel(f'PC2 ({analyzer.pca.explained_variance_ratio_[1]:.1%})')
    ax1.set_title('Enhanced PCA Clustering')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Temporal pattern heatmap
    ax2 = plt.subplot(3, 4, 2)
    
    # Create a 24-hour template
    full_day = np.zeros(24)
    for i, hour in enumerate(analyzer.actual_hours):
        full_day[hour] = clusters[i]
    
    # Reshape for heatmap
    heatmap_data = full_day.reshape(1, -1)
    
    im = ax2.imshow(heatmap_data, cmap='viridis', aspect='auto', 
                   extent=[0, 24, -0.5, 0.5])
    ax2.set_xticks(range(0, 25, 2))
    ax2.set_yticks([])
    ax2.set_xlabel('Hour of Day')
    ax2.set_title('24-Hour Cluster Pattern')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax2, orientation='horizontal', pad=0.1)
    cbar.set_label('Cluster')
    
    # 3. PC loadings heatmap
    ax3 = plt.subplot(3, 4, 3)
    
    # Top loadings for first 3 PCs
    top_loadings = loadings.iloc[:15, :3]  # Top 15 classes, first 3 PCs
    
    sns.heatmap(top_loadings.T, annot=True, fmt='.2f', cmap='RdBu_r', 
                center=0, ax=ax3, cbar_kws={'label': 'Loading'})
    ax3.set_title('PC Loadings Heatmap')
    ax3.set_xlabel('Sound Classes')
    
    # 4. Explained variance
    ax4 = plt.subplot(3, 4, 4)
    
    n_components = min(10, len(analyzer.pca.explained_variance_ratio_))
    cumvar = np.cumsum(analyzer.pca.explained_variance_ratio_[:n_components])
    
    ax4.bar(range(1, n_components + 1), 
           analyzer.pca.explained_variance_ratio_[:n_components], 
           alpha=0.7, color='skyblue', label='Individual')
    ax4.plot(range(1, n_components + 1), cumvar, 'ro-', 
            color='red', label='Cumulative')
    ax4.axhline(y=0.9, color='green', linestyle='--', label='90% threshold')
    ax4.set_xlabel('Component')
    ax4.set_ylabel('Explained Variance')
    ax4.set_title('PCA Explained Variance')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Cluster comparison radar chart
    ax5 = plt.subplot(3, 4, 5, projection='polar')
    
    # Top 8 sound classes by loading magnitude
    top_classes = loadings['PC1'].abs().nlargest(8).index
    angles = np.linspace(0, 2*np.pi, len(top_classes), endpoint=False)
    
    try:
        # Try to get mean values for each cluster from original data
        # Note: This is a simplified approach - you may need to adapt
        original_data = analyzer._reconstruct_original_data() if hasattr(analyzer, '_reconstruct_original_data') else None
        
        if original_data is not None:
            # Use original data if available
            data_to_use = original_data[top_classes]
        else:
            # Fallback: use PCA loadings as proxy
            print("Using PCA loadings as proxy for radar chart")
            data_to_use = pd.DataFrame(
                np.abs(loadings.loc[top_classes, 'PC1'].values).reshape(1, -1),
                columns=top_classes,
                index=[0]
            )
    except Exception as e:
        print(f"Warning: Using simplified radar chart. Error: {e}")
        # Simple fallback using loadings
        data_to_use = pd.DataFrame(
            np.abs(loadings.loc[top_classes, 'PC1'].values).reshape(1, -1),
            columns=top_classes,
            index=[0]
        )
    
    for cluster_id in sorted(set(clusters)):
        if len(data_to_use) > 1:
            mask = clusters == cluster_id
            cluster_means = data_to_use[mask].mean()
        else:
            # Fallback case
            cluster_means = data_to_use.iloc[0]
        
        # Normalize to 0-1 scale
        if data_to_use.max().max() > data_to_use.min().min():
            cluster_means_norm = (cluster_means - data_to_use.min().min()) / (data_to_use.max().max() - data_to_use.min().min())
        else:
            cluster_means_norm = cluster_means / cluster_means.max() if cluster_means.max() > 0 else cluster_means
        
        ax5.plot(angles, cluster_means_norm, 'o-', 
                label=f'Cluster {cluster_id}', linewidth=2)
        ax5.fill(angles, cluster_means_norm, alpha=0.1)
    
    ax5.set_xticks(angles)
    ax5.set_xticklabels([cls[:15] + '...' if len(cls) > 15 else cls 
                        for cls in top_classes], fontsize=8)
    ax5.set_title('Cluster Profiles\n(Top Sound Classes)')
    ax5.legend()
    
    # 6. Diversity indices (if available)
    if diversity_df is not None:
        ax6 = plt.subplot(3, 4, 6)
        
        colors_div = [colors[c] for c in clusters]
        ax6.scatter(diversity_df['shannon_diversity'], diversity_df['simpson_diversity'],
                   c=colors_div, s=100, alpha=0.7, edgecolors='black')
        
        ax6.set_xlabel('Shannon Diversity')
        ax6.set_ylabel('Simpson Diversity')
        ax6.set_title('Acoustic Diversity by Cluster')
        ax6.grid(True, alpha=0.3)
    
    # 7. Hierarchical clustering dendrogram
    ax7 = plt.subplot(3, 4, 7)
    
    try:
        # Perform hierarchical clustering on PCA results
        linkage_matrix = linkage(pca_results, method='ward')
        
        dendrogram(linkage_matrix, ax=ax7, 
                  labels=[f'H{h:02d}' for h in analyzer.actual_hours],
                  orientation='top')
        ax7.set_title('Hierarchical Clustering')
        ax7.set_xlabel('Hours')
    except Exception as e:
        print(f"Warning: Could not create dendrogram. Error: {e}")
        ax7.text(0.5, 0.5, 'Hierarchical clustering\nnot available\n(insufficient data)', 
                ha='center', va='center', transform=ax7.transAxes)
        ax7.set_title('Hierarchical Clustering')
    
    # 8. PC1 vs PC3 (alternative view)
    ax8 = plt.subplot(3, 4, 8)
    
    for cluster_id in sorted(set(clusters)):
        mask = clusters == cluster_id
        cluster_pca = pca_results[mask]
        
        ax8.scatter(cluster_pca.iloc[:,0], cluster_pca.iloc[:,2], 
                   c=colors[cluster_id], label=f'Cluster {cluster_id}', 
                   s=100, alpha=0.7)
    
    ax8.set_xlabel(f'PC1 ({analyzer.pca.explained_variance_ratio_[0]:.1%})')
    ax8.set_ylabel(f'PC3 ({analyzer.pca.explained_variance_ratio_[2]:.1%})')
    ax8.set_title('PC1 vs PC3')
    ax8.legend()
    ax8.grid(True, alpha=0.3)
    
    # 9-12. Individual cluster profiles
    for i, cluster_id in enumerate(sorted(set(clusters))):
        ax = plt.subplot(3, 4, 9 + i)
        
        mask = clusters == cluster_id
        cluster_hours = np.array(analyzer.actual_hours)[mask]
        
        # Show cluster in context of full day
        full_day_activity = np.zeros(24)
        for hour in cluster_hours:
            full_day_activity[hour] = 1
        
        colors_map = ['lightgray' if x == 0 else colors[cluster_id] for x in full_day_activity]
        
        ax.bar(range(24), np.ones(24), color=colors_map, alpha=0.7, edgecolor='black')
        ax.set_xlim(0, 23)
        ax.set_xlabel('Hour of Day')
        ax.set_title(f'Cluster {cluster_id} Temporal Profile')
        ax.set_xticks(range(0, 24, 4))
        
        # Add text annotation
        ax.text(0.02, 0.98, f'{len(cluster_hours)} hours', 
               transform=ax.transAxes, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# MAIN EXECUTION FUNCTION

def run_post_pca_analysis(analyzer, pca_results, clusters, loadings, session):
    """Run post-PCA analysis"""

    
    # Initialize post-PCA analyzer
    post_analyzer = PostPCAAnalyzer(analyzer, pca_results, clusters, loadings, session)
    
    # 1. Deep cluster analysis
    cluster_profiles = post_analyzer.deep_cluster_analysis()
    
    # 2. Sound class drivers
    cluster_differences = post_analyzer.analyze_driving_sound_classes()
    
    # 3. Temporal transitions
    transitions = analyze_temporal_transitions(analyzer, clusters)
    
    # 4. Acoustic diversity indices
    diversity_df = calculate_acoustic_diversity_indices(session, analyzer)
    
    # 5. Predictive modeling
    rf_model, feature_importance = build_cluster_prediction_model(pca_results, clusters, analyzer)
    
    # 6. Comprehensive visualizations
    create_comprehensive_visualizations(analyzer, pca_results, clusters, loadings, diversity_df)
    
    
    return {
        'post_analyzer': post_analyzer,
        'cluster_profiles': cluster_profiles,
        'cluster_differences': cluster_differences,
        'transitions': transitions,
        'diversity_df': diversity_df,
        'rf_model': rf_model,
        'feature_importance': feature_importance
    }

# EXECUTE POST-PCA ANALYSIS
results = run_post_pca_analysis(analyzer, pca_results, clusters, loadings, session)